# 🔗 Solana blokkanalyse – parallell uthenting via RPC

Dette prosjektet ble satt opp med ChatGPT 1. august 2025. Samtalen kan refereres her: [https://chatgpt.com/share/e/688d1ed5-04f0-800f-a579-80b2a559e77c](https://chatgpt.com/share/e/688d1ed5-04f0-800f-a579-80b2a559e77c)

GitHub-bruker: `sac-marius`

## Innhold:
- Henter de siste 1000 blokkene fra Solana
- Bruker 10 samtidige tråder (`ThreadPoolExecutor`)
- Parser ut transaksjonsdata: `timestamp`, `slot`, `from/to`, `lamports`
- Lagrer som komprimert `.parquet`

---


In [ ]:
!pip install solana pandas pyarrow tqdm


In [ ]:
from solana.rpc.api import Client
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from tqdm import tqdm
import time

client = Client("https://api.mainnet-beta.solana.com")


In [ ]:
def get_latest_slot():
    response = client.get_slot()
    return response['result']

def get_block_data(slot):
    try:
        response = client.get_block(slot, max_supported_transaction_version=0)
        result = response['result']
        if not result:
            return []

        block_time = result['blockTime']
        txs = result['transactions']
        parsed = []
        for tx in txs:
            message = tx['transaction']['message']
            meta = tx['meta']
            if not meta or not meta.get('postBalances'):
                continue

            accounts = message['accountKeys']
            from_addr = accounts[0]
            to_addr = accounts[1] if len(accounts) > 1 else None

            lamports = (
                meta['postBalances'][1] - meta['preBalances'][1]
                if len(meta['postBalances']) > 1 else 0
            )

            parsed.append({
                'slot': slot,
                'timestamp': block_time,
                'tx_signature': tx['transaction']['signatures'][0],
                'from_address': from_addr,
                'to_address': to_addr,
                'lamports': lamports
            })
        return parsed

    except Exception as e:
        print(f"⚠️ Feil på slot {slot}: {e}")
        return []


In [ ]:
start_time = time.time()

latest_slot = get_latest_slot()
n = 1000
slots = list(range(latest_slot - n, latest_slot))

results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_block_data, slot): slot for slot in slots}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Henter blokker"):
        data = future.result()
        results.extend(data)

df = pd.DataFrame(results)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

df.to_parquet("solana_blocks.parquet", index=False, compression='snappy')

total_time = time.time() - start_time
print("✅ Lagret som 'solana_blocks.parquet'")
print(f"⏱️ Total tid: {total_time:.2f} sekunder for {n} blokker")
